# Facial Recognition and Attendance

In [1]:
#importing libraries
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

In [2]:
# path to the folder which contains images of cricketers
path = 'ImagesCricketers'

In [3]:
# Initialising 2 lists which will be used to store images and names respectively
images = []
names = []

In [4]:
# list of names of images in the folder described by the path
lis = os.listdir(path)

In [5]:
# Reading images and appending them in images(list) and appending their names in names(list) 
for name in lis:
    current = cv2.imread(f'{path}/{name}')
    images.append(current)
    names.append(os.path.splitext(name)[0])

In [6]:
# This function returns a list of encodings of images
def Encode(images):
    encode_list = []
    for i in images:
        i = cv2.cvtColor(i,cv2.COLOR_BGR2RGB)
        encoded_image = face_recognition.face_encodings(i)[0]
        encode_list.append(encoded_image)
    return encode_list

In [7]:
# This function marks the attendance by taking name as argument.
# The name of the attendant and time is written in the file named "attendance.csv"
def mark_attendance(name):
    with open('attendance.csv','r+') as f:
        already_list = f.readlines()
        name_list = []
        for line in already_list:
            entry = line.split(',')
            name_list.append(entry[0])
        if name not in name_list:
            present =datetime.now()
            dtString = present.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

In [8]:
known_encoded = Encode(images) 

In [9]:
# Start capturing
capture = cv2.VideoCapture(0)

In [ ]:
# Image is read and is matched with the images in the folder
# The image in the folder which matches to the image taken by web camera is identified and the attendance is marked

while True:
    success,image = capture.read()
    imageSmall = cv2.resize(image,(0,0),None,0.25,0.25)
    imageSmall = cv2.cvtColor(imageSmall,cv2.COLOR_BGR2RGB)
    Current_frame_faces = face_recognition.face_locations(imageSmall)
    Current_frame_encodes = face_recognition.face_encodings(imageSmall,Current_frame_faces)
    
    
    for encode,loc in zip(Current_frame_encodes,Current_frame_faces):
        matches = face_recognition.compare_faces(known_encoded,encode)
        faceDistance = face_recognition.face_distance(known_encoded,encode)
        indexMatched = np.argmin(faceDistance)
        
        if matches[indexMatched]:
            identifiedName = names[indexMatched]
            y1,x2,y2,x1 = loc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(image,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(image,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(image,identifiedName,(x1+6,y2-6),cv2.FONT_HERSHEY_PLAIN,1,(255,255,255),2)
            mark_attendance(identifiedName)
#             print(identifiedName)
            
    cv2.imshow('webcam',image)
    cv2.waitKey(1)
            